# RAG Index Building

This notebook builds a simple retrieval index from the article dataset.

In [ ]:
import sys
import os
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

# Colab Setup
if 'google.colab' in sys.modules:
    !pip install -q pandas numpy scikit-learn
    if not os.path.exists('aire-researcher-sandbox'):
        !git clone https://github.com/YOUR_USERNAME/aire-researcher-sandbox.git
    %cd aire-researcher-sandbox

In [ ]:
df = pd.read_csv('data/sample_texts/articles_sample.csv')
texts = df['abstract'].fillna('')

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(texts)

# Save index (mocking a persistent vector store)
with open('tfidf_index.pkl', 'wb') as f:
    pickle.dump((vectorizer, tfidf_matrix, df), f)

print("Index built and saved to tfidf_index.pkl")